In [1]:
#!jupyter nbextension enable --py gmaps

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Store csv created in part one into a DataFrame
vacation_df = pd.read_csv('../WeatherPy/Output/weatherdata.csv') 
vacation_df

,City,Latitude,Longitude,Max Temperature (F),Humidity,Cloudiness,Wind Speed (mph)
0,Port Alfred,-33.5906,26.8910,71.37,85,57,18.63
1,Tiksi,71.6872,128.8694,9.48,90,100,9.06
2,Punta Arenas,-53.1500,-70.9167,53.60,54,75,21.85
3,Rikitea,-23.1203,-134.9692,79.36,70,8,19.80
4,Fare,-16.7000,-151.0167,83.17,71,13,21.97
...,...,...,...,...,...,...,...
531,Mayskiy,47.6931,40.1025,37.40,93,100,6.71
532,Gweta,-20.1833,25.2333,85.62,29,90,5.12
533,Cherepanovo,54.2222,83.3810,18.57,95,99,7.74
534,Umarkot,25.3616,69.7362,85.51,15,0,11.21


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps: Source: https://nu.bootcampcontent.com/NU-Coding-Bootcamp/nu-chi-data-pt-02-2021-u-c/blob/master/01-Lesson-Plans/06-Python-APIs/3/Activities/06-Evr_Jupyter_Gmaps/Solved/gmaps.ipynb
gmaps.configure(api_key=gkey)



In [13]:
# Heatmap of humidity
locations = vacation_df[["Latitude", "Longitude"]].astype(float)
weight = vacation_df["Humidity"].astype(float)

fig = gmaps.figure()

#Adds heat to layer

heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
#Adds heat layer to map
fig.add_layer(heat_layer)


 # AAdds markers to a layer 
#markers = gmaps.marker_layer(locations)
# Add marker layer to the map
#fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down cities that fit criteria and drop any results with null values


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates

#coordinate = []
#for city in  vacation_df['City']:
 #   coordinate.append = [[vacation_df['Latitude'], vacation_df['Longitude']] 
    
hotel_df = pd.DataFrame()

hotel_df['Hotel Name'] = "" #need to adjust with new DF name
hotel_df['City'] = vacation_df['City']
hotel_df['Country'] = "" #need to add
hotel_df['Latitude'] = vacation_df['Latitude']
hotel_df['Longitude'] = vacation_df['Longitude']
hotel_df['Coordinates'] = list(zip(vacation_df['Latitude'], vacation_df['Longitude']))
#hotel_df['Coordinates'] = f"{hotel_df['Latitude']}, {hotel_df['Longitude']}" #need to combine better--make function to get f sting and pass function into apply
#hotel_df.apply(f"{hotel_df['Latitude']}, {hotel_df['Longitude']}", axis=1)
hotel_df.head()

,Hotel Name,City,Country,Latitude,Longitude,Coordinates
0,NaN,Port Alfred,,-33.5906,26.8910,"(-33.5906, 26.891)"
1,NaN,Tiksi,,71.6872,128.8694,"(71.6872, 128.8694)"
2,NaN,Punta Arenas,,-53.1500,-70.9167,"(-53.15, -70.9167)"
3,NaN,Rikitea,,-23.1203,-134.9692,"(-23.1203, -134.9692)"
4,NaN,Fare,,-16.7000,-151.0167,"(-16.7, -151.0167)"


In [12]:
# Set parameters to search for a hotel
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
target_radius = 5000
target_type = "hotel"
params = {
     
    #"rankby": "distance",
    "radius": target_radius,
    "type": target_type,
    "key": gkey
    }

# Iterate through 
for index, row in hotel_df.iterrows():

    # get lat, lng from df
    location = hotel_df["Coordinates"]
    # Use the search term: "Hotel" and our lat/lng
    #hotel = row['Hotel Name']
    #params['Hotel Name'] = hotel
    params['location'] = location

    # make request and print url
    print(f"Retrieving Results for {index}: {location}.")
    
    
    # convert to json
   
    response = requests.get(base_url, params=params).json()
    results = response['results']
    # Grab the first hotel from the results and store the name
    try:
        print(f"Closest {hotel} hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, 'Country'] = results[0]['country']
   
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    


Retrieving Results for 0: 0         (-33.5906, 26.891)
1        (71.6872, 128.8694)
2         (-53.15, -70.9167)
3      (-23.1203, -134.9692)
4         (-16.7, -151.0167)
               ...          
531       (47.6931, 40.1025)
532      (-20.1833, 25.2333)
533        (54.2222, 83.381)
534       (25.3616, 69.7362)
535         (-1.7, -77.9333)
Name: Coordinates, Length: 536, dtype: object.


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
